## Base Models

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
pos_final_test = pd.read_csv('./pos_final_test.csv.gz', compression='gzip')
pos_final_test_KNN = pd.read_csv('./pos_KNN_final_test.csv.gz', compression='gzip')
pos_final_missing_data_test = pd.read_csv('./pos_final_missing_data_test.csv.gz', compression='gzip')


In [3]:
model_cols = ~pos_final_test.columns.isin(['(\'subject_id\', \'\')', '(\'hadm_id\', \'\')', 'subject_id', 'hadm_id', 'Unnamed: 0', 'pos'])

hadm_features = pos_final_test.loc[:, model_cols]
hadm_target = pos_final_test.loc[:, 'pos']

# hadm_features = pos_final_missing_data_test.loc[:, model_cols]
# hadm_target = pos_final_missing_data_test.loc[:, 'pos']


In [4]:
# # Creatinine - 50912
# hadm_features = hadm_features.loc[:, ['(\'max\', 50912)',  '(\'above_max\', 50912)', '(\'mean\', 50912)',  '(\'abn_percent\', 50912)']]

# # Least Missingness
# # [51221, 51265, 51222, 51301, 51279, 51277, 51248, 51249, 51250]
# hadm_features = hadm_features.loc[:, ['(\'max\', 51221)',  '(\'above_max\', 51221)', '(\'mean\', 51221)',  '(\'abn_percent\', 51221)',
#                                         '(\'max\', 51265)',  '(\'above_max\', 51265)', '(\'mean\', 51265)',  '(\'abn_percent\', 51265)',
#                                         '(\'max\', 51222)',  '(\'above_max\', 51222)', '(\'mean\', 51222)',  '(\'abn_percent\', 51222)',
#                                         '(\'max\', 51301)',  '(\'above_max\', 51301)', '(\'mean\', 51301)',  '(\'abn_percent\', 51301)',
#                                         '(\'max\', 51279)',  '(\'above_max\', 51279)', '(\'mean\', 51279)',  '(\'abn_percent\', 51279)',
#                                         '(\'max\', 51277)',  '(\'above_max\', 51277)', '(\'mean\', 51277)',  '(\'abn_percent\', 51277)',
#                                         '(\'max\', 51248)',  '(\'above_max\', 51248)', '(\'mean\', 51248)',  '(\'abn_percent\', 51248)',
#                                         '(\'max\', 51249)',  '(\'above_max\', 51249)', '(\'mean\', 51249)',  '(\'abn_percent\', 51249)',
#                                         '(\'max\', 51250)',  '(\'above_max\', 51250)', '(\'mean\', 51250)',  '(\'abn_percent\', 51250)']]

# # Least Missingness + Creatinine
# hadm_features = hadm_features.loc[:, ['(\'max\', 51221)',  '(\'above_max\', 51221)', '(\'mean\', 51221)',  '(\'abn_percent\', 51221)',
#                                         '(\'max\', 51265)',  '(\'above_max\', 51265)', '(\'mean\', 51265)',  '(\'abn_percent\', 51265)',
#                                         '(\'max\', 51222)',  '(\'above_max\', 51222)', '(\'mean\', 51222)',  '(\'abn_percent\', 51222)',
#                                         '(\'max\', 51301)',  '(\'above_max\', 51301)', '(\'mean\', 51301)',  '(\'abn_percent\', 51301)',
#                                         '(\'max\', 51279)',  '(\'above_max\', 51279)', '(\'mean\', 51279)',  '(\'abn_percent\', 51279)',
#                                         '(\'max\', 51277)',  '(\'above_max\', 51277)', '(\'mean\', 51277)',  '(\'abn_percent\', 51277)',
#                                         '(\'max\', 51248)',  '(\'above_max\', 51248)', '(\'mean\', 51248)',  '(\'abn_percent\', 51248)',
#                                         '(\'max\', 51249)',  '(\'above_max\', 51249)', '(\'mean\', 51249)',  '(\'abn_percent\', 51249)',
#                                         '(\'max\', 51250)',  '(\'above_max\', 51250)', '(\'mean\', 51250)',  '(\'abn_percent\', 51250)',
#                                      '(\'max\', 50912)',  '(\'above_max\', 50912)', '(\'mean\', 50912)',  '(\'abn_percent\', 50912)']]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(hadm_features, hadm_target, stratify=hadm_target, random_state=0)


## Logistic Regression

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [7]:
%%time

# L2
params_log = [ {'C':np.logspace(-4, 3, 20)} ]
logReg = LogisticRegression(max_iter = 2000, class_weight = "balanced")

# # L1
# params_log = [ {'C':np.logspace(-6, -1, 20)} ]
# logReg = LogisticRegression(penalty = 'l1', max_iter = 2000, class_weight = "balanced", solver = 'liblinear')

grid_log = GridSearchCV(estimator = logReg, param_grid = params_log, cv = 5)
grid_log.fit(X_train, y_train)
grid_log.best_score_

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

CPU times: user 4min 15s, sys: 34.5 s, total: 4min 49s
Wall time: 49.8 s


0.8249678294641349

In [8]:
grid_log.best_params_

{'C': 0.00023357214690901214}

In [9]:
est_coefs = pd.Series(grid_log.best_estimator_.coef_[0], index=X_train.columns)

sortedCoefs = np.abs(est_coefs).sort_values(ascending=False)
sortedCoefs[sortedCoefs > 0]

age                                        0.137544
('abn_percent', 51006)                     0.103972
('abn_percent', 51003)                     0.093135
('min', 51006)                             0.079919
('mean', 51006)                            0.076955
                                             ...   
('min', 50882)                             0.000205
('max', 50960)                             0.000187
('max', 51514)                             0.000136
('mean', 51478)                            0.000133
ethnicity_AMERICAN INDIAN/ALASKA NATIVE    0.000010
Length: 233, dtype: float64

In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [11]:
pred = grid_log.predict(X_test)

print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
print("_______________________________________________")
print("Classification Report:", end='')
print(f"\tPrecision Score: {precision_score(y_test, pred) * 100:.2f}%")
print(f"\t\t\tRecall Score: {recall_score(y_test, pred) * 100:.2f}%")
print(f"\t\t\tF1 score: {f1_score(y_test, pred) * 100:.2f}%")
print("_______________________________________________")
print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

Accuracy Score: 80.78%
_______________________________________________
Classification Report:	Precision Score: 38.97%
			Recall Score: 70.67%
			F1 score: 50.24%
_______________________________________________
Confusion Matrix: 
 [[1553  332]
 [  88  212]]



## Random Forest

In [12]:
from sklearn import ensemble
import sklearn.model_selection as ms

randomForest = ensemble.RandomForestClassifier(class_weight = "balanced")

In [13]:
%%time

grid_para_forest = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 20),
    'n_estimators': range(10, 110, 10)
}

grid_search_forest = ms.GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', cv=5, n_jobs=-1)
grid_search_forest.fit(X_train, y_train)

CPU times: user 13.4 s, sys: 878 ms, total: 14.3 s
Wall time: 4min 46s


GridSearchCV(cv=5, estimator=RandomForestClassifier(class_weight='balanced'),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 20),
                         'n_estimators': range(10, 110, 10)},
             scoring='accuracy')

In [14]:
# Get most important features
tree_final = grid_search_forest.best_estimator_
feature_importance = tree_final.feature_importances_
feature_importance = list(zip(X_train.columns, feature_importance))

feature_importance.sort(key = lambda x: x[1], reverse = True)
feature_importance

[("('max', 51006)", 0.0406135450754391),
 ("('mean', 51006)", 0.033672623910801225),
 ('age', 0.031932670028243036),
 ("('abn_percent', 51006)", 0.02804365999472692),
 ("('above_max', 51006)", 0.02530021176836792),
 ("('mean', 50912)", 0.021775067479577644),
 ("('max', 50912)", 0.0214793307418732),
 ("('min', 51003)", 0.02057036924765477),
 ("('mean', 51277)", 0.01536572269687816),
 ("('abn_percent', 50912)", 0.0151375189000434),
 ("('above_max', 50912)", 0.01379264022579788),
 ("('min', 51249)", 0.013233695153579365),
 ("('max', 51277)", 0.012548032189664379),
 ("('mean', 51003)", 0.01148201706269082),
 ("('mean', 51249)", 0.011423342976348148),
 ("('min', 51006)", 0.011100377540062161),
 ("('abn_percent', 51003)", 0.010927649857030602),
 ("('max', 51237)", 0.010896009334052777),
 ("('mean', 51237)", 0.01080149231747837),
 ("('mean', 51275)", 0.010729597941442523),
 ("('min', 51277)", 0.010523181493030102),
 ("('min', 51274)", 0.010075129875343128),
 ("('max', 51003)", 0.0100553482569

In [15]:
grid_search_forest.best_params_

{'criterion': 'entropy', 'max_depth': 14, 'n_estimators': 80}

In [16]:
grid_search_forest.best_score_

0.8927206665851486

In [17]:
print("The training error is: %.5f" % (grid_search_forest.score(X_train, y_train)))
print("The test     error is: %.5f" % (grid_search_forest.score(X_test, y_test)))

The training error is: 0.99924
The test     error is: 0.87872


In [18]:
rf_predictions = tree_final.predict(X_test)

print(f"Accuracy Score: {accuracy_score(y_test, rf_predictions) * 100:.2f}%")
print("_______________________________________________")
print("Classification Report:", end='')
print(f"\tPrecision Score: {precision_score(y_test, rf_predictions) * 100:.2f}%")
print(f"\t\t\tRecall Score: {recall_score(y_test, rf_predictions) * 100:.2f}%")
print(f"\t\t\tF1 score: {f1_score(y_test, rf_predictions) * 100:.2f}%")
print("_______________________________________________")
print(f"Confusion Matrix: \n {confusion_matrix(y_test, rf_predictions)}\n")

Accuracy Score: 87.87%
_______________________________________________
Classification Report:	Precision Score: 61.59%
			Recall Score: 31.00%
			F1 score: 41.24%
_______________________________________________
Confusion Matrix: 
 [[1827   58]
 [ 207   93]]

